In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0


# Credit Card Churn Analysis
---  

## Project Description 
In this project, I will be working with a dataset provided by **Kaggle**, where I will develop a churn-rate analysis. The goal is to identify the causes and reasons for customer churn from a banking institution in relation to credit card services. After understanding these causes and reasons, some machine learning models will be developed to predict potential customers who will be abandoning the credit card service of this institution. With these predictions, I will seek to develop solutions to prevent or reverse the churn of these customers.  

---  

### CRISP-DM Methodology  
The project will follow the CRISP-DM (*Cross-Industry Standard Process for Data Mining*) framework:  

| **Stage** | **Objective** | **Key Actions** |  
|-----------|---------------|------------------|  
| **1. Business Understanding** | Define the impact of churn prediction on customer retention. | - Identify costs of false negatives.<br>- Align metrics with business KPIs. |  
| **2. Data Understanding** | Analyze data structure, quality, and variable relationships. | - Exploratory Data Analysis (EDA).<br>- Outlier and correlation detection. |  
| **3. Data Preparation** | Prepare data for model training. | - Split training and test data.<br>- Remove redundant variables. |  
| **4. Modeling** | Train and compare classical models and neural networks. | - Random Forest/Logistic Regression (baseline).<br>- PyTorch neural network (focus on generalization). |  
| **5. Evaluation** | Validate performance with business-oriented metrics. | - AUC-ROC, confusion matrix.<br>- Simulate financial impact. |  
| **6. Deployment** | Deploy the model for production use. | - Build a final churn prediction model with customer behavior indicators. |  

*This notebook covers the Business Understanding, Data Understanding, and Data Preparation.*  

---  


In [0]:
%pip install seaborn==0.13.2

## Imports:


In [0]:
# Data analizing/preparation:
# PySpark.SQL
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import NumericType
# Numpy 
import numpy as np
# Pandas
import pandas as pd
# Sklearning features
from sklearn.feature_selection import chi2
# Scipy
from scipy.stats import ttest_ind

# Graphics:
# Matplotlib
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns

# SRC/ Functions Utils
import sys
sys.path.append('/Workspace/Repos/otniel.g.andrade@outlook.com/1_Portfolio-Credit-Card_Churn_Analysis_with_Pytorch/src')
from data_manipulation import DataSpark
from visualization import GraphicsData


## 1 - Business Understanding  
---

### General Problem Context  
#### What is Churn Rate, and What Are the Solutions to This Problem? 
Many companies struggle with customer churn and often find it challenging to reverse this trend. The metric that measures this scenario is called **churn rate**, which indicates when strategic solutions are needed to address the issue.  

In 2020, Bryce Baer published a guide on churn rate on the [Zendesk website](https://www.zendesk.com.br/blog/customer-churn-rate/?_ga=2.155312252.614584228.1623244699-1365810980.1622555740#) – a company specializing in corporate software development. The guide highlights that businesses implementing strategies to reduce churn can increase their **profitability** by nearly 40%.  

---

#### How to Calculate Churn Rate?
##### Churn Rate Formula:  
$$\text{Churn Rate} = \frac{\text{Number of customers lost during a period}}{\text{Total number of customers at the start of the period}} \times 100$$  

---

#### Impacts of a High Churn Rate
While reducing churn to zero is practically impossible, acceptable rates (4% to 5%) minimize financial impacts. Some companies operate at higher rates (5% to 7%) without significant revenue loss, depending on industry dynamics. **Key factors to define "acceptable" churn**:  
- Industry standards (e.g., SaaS vs. retail).  
- Customer lifetime value (CLV).  
- Customer acquisition cost (CAC).  

---

#### Reasons for Customer Churn
1. **Lack of Perceived Value**:  
   - Occurs when there’s a growing gap between customer expectations and actual delivery. Clear communication about product/service benefits is critical.  
2. **Poor Customer Experience**:  
   - Negative interactions (e.g., bad support, complex processes, product failures) drive churn.  
3. **Competitor Offers**:  
   - Attractive promotions or pricing from competitors can lure customers away.  
4. **Changing Customer Needs**:  
   - Failure to adapt products/services to evolving demands leads to turnover.  

---

## Project Challenge: 
The bank’s manager has observed a rising number of customers abandoning credit card services. Stakeholders aim to:  
1. **Analyze historical data** to identify root causes of churn.  
2. **Develop a machine learning model** to predict customer churn probability.  
3. **Implement strategic actions** to retain high-risk customers.  

---

## KPIs for the Churn Prediction Project:  
1. **Churn Rate**:  
   - *Definition*: Percentage of customers who discontinue credit card services within a specific period.  
   - *Goal*: Reduce this metric through targeted retention strategies.  

2. **Retention Rate**:  
   - *Definition*: Percentage of customers retained after a period.  
   - *Importance*: Directly reflects the success of retention efforts.  

3. **Customer Acquisition Cost (CAC) vs. Retention Cost**:  
   - *Definition*: Ratio of costs to acquire new customers vs. retaining existing ones.  
   - *Insight*: Retention is typically **5-7x cheaper** than acquisition.  

4. **AUC-ROC (Area Under the Receiver Operating Characteristic Curve)**:  
   - *Definition*: Measures the model’s ability to distinguish between churners and non-churners.  
   - *Target*: AUC-ROC > 0.85.  

5. **Recall**:  
   - *Definition*: Proportion of actual churners correctly identified by the model.  
   - *Importance*: High recall ensures fewer **false negatives** (missed churners), which is critical because a false negative could result in losing a customer. Retaining existing customers through targeted strategies is significantly cheaper than acquiring new ones.  

---



## 2 - Data Understanding

---

* This dataset consists of 10,000 customers mentioning their age, salary, marital_status, credit card limit, credit card category, etc.

---

- **Data file**: - BankChurners.csv

---

- **Target dependent variable**: - 'Attrition_Flag', categorical column with binary classification, i.e. 'Existing Customer'(No-churner) or 'Attrited Customer'(Churner).

---

- **The dataset colleted from kaggle**: https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers?sort=votes&select=BankChurners.csv

---
- **The dataset origin from this site**: https://leaps.analyttica.com/home

---



### Loading Dataset

I will use Medallion Architecture to organize and classify the data. This approach organizes the data into different levels of processing and refinement, making it easier to manage and analyze the data. Here is a summary of each layer:

---

- **Bronze Layer:**

  Description: Stores the raw data, exactly as it was collected from different sources.
  
  Objective: Preserve the integrity of the original data, without any transformation.
---
- **Silver Layer:**

  Description: Contains pre-processed and cleaned data.
  
  Objective: Perform basic transformations, such as data cleaning, standardization, and type correction.
---
- **Gold Layer:**

  Description: Stores the refined data, ready for analysis and final consumption.
  
  Objective: Apply specific corrections and improvements according to business needs.

---

### Bronze Data Tier

In [0]:
# Creating a directory to store the files
dbutils.fs.mkdirs('/Volumes/workspace/projects-data-science/churn-project-data/bronze')
###### >>>>>>> Note: At this point, upload the files present in the notebook repository folder to this directory

# Viewing the location of files
display(dbutils.fs.ls('/Volumes/workspace/projects-data-science/churn-project-data/bronze'))

In [0]:
# File location and type
file_location = '/Volumes/workspace/projects-data-science/churn-project-data/bronze/BankChurners.csv'
file_type = 'csv'
# Load Data
df_csv = DataSpark(spark= spark, file_location = file_location).load_data(file_type = file_type)
# Show Data
df_csv.limit(10).display()

### Saving dataset in Parquet format for more performance at consultations

In [0]:
# File location and type
# Creating a directory to store the files
file_location = '/Volumes/workspace/projects-data-science/churn-project-data/bronze/parquet'
file_type = 'parquet'

# Save Data
DataSpark(spark = spark, dataframe = df_csv, file_location = file_location).save_data(file_type = file_type)
# Load Data
df = DataSpark(spark = spark, file_location = file_location).load_data(file_type = file_type)
# Show Data
df.limit(10).display()

### Dictionary of Dataset
---

**CLIENTNUM**: Client number. Unique identifier for the customer holding the account

---

**Attrition_Flag**: Internal event (customer activity) variable - if the account is closed then 'Attrited Customer' else 'Existing Customer'

---

**Customer_Age**: Demographic variable - Customer's Age in Years

---

**Gender**: Demographic variable - M=Male, F=Female

---

**Dependent_count**: Demographic variable - Number of dependents

---

**Education_Level**: Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.)

---

**Marital_Status**: Demographic variable - Married, Single, Divorced, Unknown

---

**Income_Category**: Demographic variable - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, >$120K)

---

**Card_Category**: Product Variable - Type of Card (Blue, Silver, Gold, Platinum)

---

**Months_on_book**: Period of relationship with bank

---

**Total_Relationship_Count**: Total no. of products held by the customer

---

**Months_Inactive_12_mon**: No. of months inactive in the last 12 months

---

**Contacts_Count_12_mon**: No. of Contacts in the last 12 months

---

**Credit_Limit**: Credit Limit on the Credit Card

---

**Total_Revolving_Bal**: Total Revolving Balance on the Credit Card

---

**Avg_Open_To_Buy**: Open to Buy Credit Line (Average of last 12 months)

---

**Total_Amt_Chng_Q4_Q1**: Change in Transaction Amount (Q4 over Q1)

---

**Total_Trans_Amt**: Total Transaction Amount (Last 12 months)

---

**Total_Trans_Ct**: Total Transaction Count (Last 12 months)

---

**Total_Ct_Chng_Q4_Q1**: Change in Transaction Count (Q4 over Q1)

---

**Avg_Utilization_Ratio**: Average Card Utilization Ratio

---


### The Size Dataset

In [0]:
print(f'Number of registers: {df.count()}\nNumber of columns: {len(df.columns)}')

### Drop Redundantes Columns:

According to the documentation of this dataset that was made available by Kaggle, we were given the recommendation to remove the columns:

---
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1

---
- Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2

---
Therefore, I will be applying this recommendation.

Will also be removing the **CLIENTNUM** column, which refers to the registration number of the customers of this banking institution. It is possible to conclude that this data will not add any relevant information to the resolution of the problems and questions to be answered with this analysis.

---

In [0]:
redundants_cols = [
  'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
  'CLIENTNUM'
]
df = df.drop(* redundants_cols)

# Check size Dataset
df.count(), len(df.columns)

### Checking data and its characteristics

In [0]:
df.printSchema()

#### Checking for null data

In [0]:
# Size df
size_df = df.count()

# Check null data
df.agg(*[F.round(((F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)) / size_df) * 100), 2).alias(c) for c in df.columns]) \
    .display()

In [0]:
# Size df
size_df = df.count()

# List of results
null_percents = []

for c in df.columns:
    if isinstance(df.schema[c].dataType, NumericType):
        # For numerics: checks for NaN and null
        null_count = F.count(F.when(F.isnan(c) | F.col(c).isNull(), c))
        
    else:
        # For non-numerics: check only for null
        null_count = F.count(F.when(F.col(c).isNull(), c))
    
    null_percents.append(
        F.round((null_count / size_df) * 100, 2).alias(c)
    )

df.agg(*null_percents).display()

#### Checking data duplicate 

In [0]:
df.groupby(df.columns) \
    .count() \
    .filter(F.col('count') > 1) \
    .display()

### Classifying variables
#### Concepts for Classification of variables according to statistics:

---

**Quantitative or numerical variables**:

* *Discrete*: only take integer values

* *Continuous*: assumes any value in the range of real numbers

---

**Qualitative or categorical variables**:
* *Nominals*: when categories do not have a natural order

* *Ordinals*: when categories can be ordered.

---

##### Adjusting column names

In [0]:
for column in df.columns:
    
    # Renaming columns with only lowercase letters    
    df = df.withColumnRenamed(column, column.lower()) \

df.limit(5).display()       

In [0]:
# Numerical Variables

discrete_numerical = [
    'customer_age', 'dependent_count', 'months_on_book', 'total_relationship_count', 'months_inactive_12_mon', 'contacts_count_12_mon', 'total_trans_ct'
    ]

continuos_numerical = [
    'credit_limit','total_revolving_bal', 'avg_open_to_buy', 'total_amt_chng_q4_q1', 'total_trans_amt', 'total_ct_chng_q4_q1',
    'avg_utilization_ratio'

    ]


# Categorical Variables

nominal_categorical = [
    'attrition_flag', 'gender', 'marital_status', 
    ]

ordinal_categorical  = [
    'education_level', 'income_category', 'card_category', 

    ]


In [0]:
# Create dataset with column types
column = ['ct_type_cols']
data = [ (name_col, ) for name_col in df.columns]
type_columns = spark.createDataFrame(data, column)

# Adding types
type_columns = type_columns \
  .withColumn('ct_type_cols', F.when(F.col('ct_type_cols').isin(nominal_categorical), 'Nominal Categorical').otherwise(F.col('ct_type_cols'))) \
  .withColumn('ct_type_cols', F.when(F.col('ct_type_cols').isin(ordinal_categorical), 'Ordinal Categorical').otherwise(F.col('ct_type_cols'))) \
  .withColumn('ct_type_cols', F.when(F.col('ct_type_cols').isin(continuos_numerical), 'Continuos Numerical').otherwise(F.col('ct_type_cols'))) \
  .withColumn('ct_type_cols', F.when(F.col('ct_type_cols').isin(discrete_numerical), 'Discrete Numerical').otherwise(F.col('ct_type_cols'))) \

# Colleting data
groupy_type_columns = type_columns.groupBy('ct_type_cols') \
  .agg(F.count('ct_type_cols').alias('count_types')) \
  .withColumn('percentage', F.round((F.col('count_types') / len(df.columns)) * 100, 2)) \
  .orderBy('count_types') \

# Graphic
# Data
data_ax = groupy_type_columns.toPandas()

In [0]:
pie_type_var = GraphicsData(data = data_ax)
pie_type_var.plot_variable_type(count_col='count_types', label_col='ct_type_cols')

### Checking the data initially to verify its characteristics and structure

####Checking some statistical data from the numerical columns of the data

In [0]:
df_describe = df.select(*discrete_numerical, *continuos_numerical) \
    .describe()
df_describe.display()

#### Mean of numerical variables

In [0]:
df_describe.filter(F.col('summary') == 'mean').display()

#### Check categorial variables

In [0]:
# Iterating over dataset columns
for column in  df.select(*nominal_categorical, *ordinal_categorical).columns:

    # Grouping columns by frequency and percentage
    df.groupBy(column) \
        .agg(F.count(column).alias('frequency')) \
        .withColumn('percentage', F.round((F.col('frequency') / df.count()) * 100, 2)) \
        .orderBy('frequency', ascending = False) \
        .display()     

#### Checking Outliers

The min values ​​of numerical data

In [0]:
df_describe.filter(F.col('summary') == 'min').display()

The max values ​​of numerical data

In [0]:
df_describe.filter(F.col('summary') == 'max').display()

Distribution of numerical variables

In [0]:
# Data Collect
data_ax = df.select(*discrete_numerical, *continuos_numerical).toPandas()
# Histoplots of Numerical Variables
GraphicsData(data_ax).numerical_histograms()

In [0]:
# Boxplots of Numerical Variables
GraphicsData(data_ax).numerical_boxplots(showfliers = True)